# Simulated Annealing 

## 1.
Solve for the Knapsack Problem using simulated annealing: given a finite amount of objects in wich each one has a wieght and value. Find the subset of objects that can be introduced to the backpack maximizing the value of the objects and minimizing the weight. 

The input format will be perceibed as:

**Example**

100 0.1 ---> Initial and final temperature

5 ----> amount of objects

15 ----> maximum capacity C of the knapsack

5 2 ---> (value of object, weight of object)

14 3

7 7

2 5

23 10

## 2. 
Get:
1. Best solution considering M executions: (list of objects, binary string, f-value)
2. Worst solution considering M executions: (list of objects, binary string, f-value)
3. Solution of the median of the M executions: (list of objects, binary string, f-value)
4. Mean f-value considering M executions
5. Standard deviation of target function considering the M executions


## Bonus: 
Change the Simulated Annealing Design and try to make it better. 

In [300]:
import random
from math import exp
import statistics
import math
import numpy as np

In [239]:
def g(x,w,c):
    #Returning the constraint value and it it is bellow or equal the constraint.
    flag = True
    g = 0
    
    for i in range(len(x)):
        if g<=c:
            g += (x[i]*w[i])
            if g>c:
                flag = False
                return g, flag
        else:
            flag = False   
            return g, flag
    return g,flag

In [107]:
print(g(x0, [4,5,3,2,1],6))
x0

(9, False)


[1, 1, 1, 0, 0]

In [2]:
def T(t):
    return 0.99*t

In [63]:
def X_0(W, C):
    x = [0 for i in range(len(W))]
    g = 0
    indices = [i for i in range(len(W))]
    
    while g<=C:
        aux = random.choice(indices) #choosing wich element will be in the backpack
        indices.remove(aux) #don't want infinite loops. Once in, you can't take it out of the backpack
        x[aux] = 1
        aux2 = g + W[aux]
        if aux2 <=C: #Check that it only adds if I get less than C 
            g += W[aux]
        else:
            break
    return x

In [64]:
x0 = X_0([0,1,2,3,4],6)
x0

[0, 0, 0, 1, 1]

In [118]:
def N(X, W, C):
    #Returning neighborhood with only feasible neighbors.
    N = []
    Vecindario = []
    for i in range(len(X)):
        Vecindario.append(X.copy()) #I don't want to modify my solution

    for i in range(len(Vecindario)): #Generating the whole neighborhood
        get_ele = Vecindario[i][i]
        if get_ele == 0:
            Vecindario[i][i] = 1
        elif get_ele == 1:
            Vecindario[i][i] = 0
            
    for i in Vecindario: 
        #Just taking the legal options for the constraint. 
        #This part is what ensures me that I will only have real and feasible answers
        G, flag = g(i,W,C)
        if flag == True:
            N.append(i)
    return N

In [119]:
N(x0, [1,2,3,4,5],6)

[[0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0]]

In [116]:
x0

[1, 1, 1, 0, 0]

In [262]:
def Simulated_annealing(temp, X, C):
    def f(x, p): #This had to go in: Python was having trouble reading it outside
        f = 0
        for i in range(len(x)):
            f += (p[i]*x[i])   
        return f
    
    
    tempera = temp[0]
    #t = 0 ---> could be used to change the Temperature function.
    values = [i[0] for i in X]
    weights = [i[1] for i in X]
    
    x0 = X_0(weights, C)
    xbest = x0
    fbest = f(x0,values)
    
    def auxiliar_exp(selec,fbest, temperatura): 
        #this is for theconditional inside the loop
        num = 0-(selec - fbest)
        den = T(temperatura)
        return exp(num/den)
    
    while tempera > temp[1]:
        neig = N(x0,weights,C) #Generating feasible neighbors
        selec = random.choice(neig) #Choosing a random neighbor
        if f(selec,values) <= fbest:
            xbest = selec
            fbest = f(selec,values) 
        else:
            if random.choice([0,1]) < auxiliar_exp(f(selec,values),fbest,tempera):
                xbest = selec 
                fbest = f(selec,values)
        tempera = T(tempera)
        #t += 1
    
    G,flag = g(xbest,weights,C)
    return xbest,fbest, G

In [248]:
Simulated_annealing([1000,0.1], [[5.0, 2.0],[14.0,3.0],[7.0,7.0],[2.0,5.0],[23.0,10.0]],15.0)

[1, 0, 0, 1, 1]


([1, 0, 0, 1, 0], 7.0, 7.0)

In [237]:
def representation(X):
    #Generating the string asociated to the list of objects
    #Returns a list of the objects that are inside the knapsack
    cadena = ""
    posiciones = []
    for i in range(len(X)):
        cadena += str(X[i])
        if X[i] ==1:
            posiciones.append(i)
    return cadena, posiciones 

In [205]:
representation([0, 1, 0, 0, 1])

('01001', [1, 4])

# Part one: just visualizing

Print the solution with format:

list_of_positions string_of_solution
F-score
G-Score

In [291]:
def Printing(Temperature, Data, Max_C):
    Xbest, fbest, G = Simulated_annealing(Temperature, Data, Max_C)
    String, Positions = representation(Xbest)
    
    print(Positions, String)
    print(fbest)
    print(G)
    
    return Positions, String, fbest, G

In [292]:
Printing([1000,0.1], [[5.0, 2.0],[14.0,3.0],[7.0,7.0],[2.0,5.0],[23.0,10.0]],15.0)

[0, 1, 2] 11100
26.0
12.0


([0, 1, 2], '11100', 26.0, 12.0)

# Part two:

Implement the M exections of your algorithm to solve an instance of the problem stored in a file with the parameters shown at the beginning. 

In [314]:
if __name__=='__main__':
    filename = str(input("Filename: "))
    M = int(input("M executions: "))
    
    def Read(filename):
        file1 = open(filename, 'r')
        Lines = file1.readlines()
        
        temperaturas = list(map(float,Lines[0].split())) #Temperature
        Objetos = int(Lines[1]) #Amount of objects
        Capacidad = float(Lines[2]) #Max knapsack capacity
        A = []
        for i in range(3, Objetos+3): #skipping the first three lines read
            A.append((list(map(float,Lines[i].split()))))
        return temperaturas, Capacidad, A
    
    temperaturas, Capacidad, Datos = Read(filename)
    
    solutions = {}
    for i in range(M):
        x, f, G = Simulated_annealing(temperaturas, Datos, Capacidad)
        X, posiciones= representation(x)
        solutions[X] = [f,G, posiciones]
    
    G_values = [i[1] for i in solutions.values()]
    F_values = [i[0] for i in solutions.values()]
    Position_answers = [i[2] for i in solutions.values()]
    Strings = [i for i in solutions.keys()]
    
    #This has to be first: point 1,2 and 3 need sorting the dictionary
    median_key = math.ceil(statistics.median([i for i in range(M)]))
    Median = [G_values[median_key],F_values[median_key], Position_answers[median_key],Strings[median_key]]
    
    mean_F = sum(F_values)/float(M)
    standard_deviations_F = np.std(np.array(F_values))
    
    Solutions_sorted = sorted(solutions.items(), key=lambda x: x[1])   
    Best_solution = Solutions_sorted[-1]
    Worst_solution = Solutions_sorted[0]
    
    print("Best solution in ", M, " iterations: \nObjects:", Best_solution[1][2], 
          "String:", Best_solution[0], "\n Value of F and G respectively: ", Best_solution[1][0], Best_solution[1][1])
    print("\n")
    print("Worst solution in ", M, " iterations: \nObjects:", Worst_solution[1][2], 
          "String:", Worst_solution[0], "\n Value of F and G respectively: ", Worst_solution[1][0], Worst_solution[1][1])
    print("\n")
    print("Solution of median in: ", M, " iterations \nObjects:", Median[2], "String:", Median[3], 
          "\n Value of F and G respectively: ", Median[1], Median[0])
    print("\n")
    print("Mean solution of target function: ", mean_F)
    print("\n")
    print("Standard deviation of target function: ", standard_deviations_F)  

    

Filename: test2.txt
M executions: 5
Best solution in  5  iterations: 
Objects: [0, 1, 4] String: 11001 
 Value of F and G respectively:  42.0 15.0


Worst solution in  5  iterations: 
Objects: [1, 3] String: 01010 
 Value of F and G respectively:  16.0 8.0


Solution of median in:  5  iterations 
Objects: [1, 4] String: 01001 
 Value of F and G respectively:  37.0 13.0


Mean solution of target function:  28.4


Standard deviation of target function:  9.728309205612248
